In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Ridge

import os
os.chdir('/Users/liyuan/desktop/SI699/codes')

In [3]:
from data_preprocess import Data_preprocess
from model_pipeline import modeling_pipeline

In [5]:
# load sampled data (10000 exampless)
data = pd.read_csv('../res/cleaned_sampled_data_10000.csv')
data.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,new_comp5_rate_percent_diff,new_comp6_rate,new_comp6_inv,new_comp6_rate_percent_diff,new_comp7_rate,new_comp7_inv,new_comp7_rate_percent_diff,new_comp8_rate,new_comp8_inv,new_comp8_rate_percent_diff
0,664572,2012-11-01 00:48:02,24,216,3.375,154.12,99,84066,3,3.0,...,0,0,0,0,1,1,1,0,0,0
1,583733,2012-11-01 03:18:56,15,117,3.375,154.12,202,29484,5,5.0,...,0,0,0,0,0,0,0,0,0,0
2,649019,2012-11-01 05:47:36,5,158,3.375,154.12,219,1840,3,4.5,...,0,0,0,0,0,0,0,0,0,0
3,571848,2012-11-01 06:46:18,5,219,3.375,154.12,219,33331,3,4.0,...,0,0,0,0,0,0,0,0,1,1
4,57094,2012-11-01 07:47:43,5,219,3.375,154.12,219,28930,3,4.0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
variables = [col for col in data.columns.unique().tolist() if col not in ['price_usd','date_time']]
model_p = modeling_pipeline(data, Ridge(), variables)
train_data, val_data, test_data = model_p.split_data()
X_train, y_train, X_val, y_val, X_test, y_test = model_p.get_X_y()
categorical_vars, categorical_binary_vars, continous_vars = model_p.divide_variables()

# TODO: check why validation RMSE is this large
model_p.get_modeling_result()

training size: 6400
validation size: 1600
test size: 2000
categorical binary vars:  50
categorical non binary vars:  7
continues vars:  29
training RMSE: 201.09516302328467
valiation RMSE: 18262.218090826274


(201.09516302328467, 18262.218090826274)

In [7]:
### ridge with continuous variables
variables = continous_vars
model_p = modeling_pipeline(data, Ridge(), variables)
train_data, val_data, test_data = model_p.split_data()
X_train, y_train, X_val, y_val, X_test, y_test = model_p.get_X_y()
categorical_vars, categorical_binary_vars, continous_vars = model_p.divide_variables()

# TODO: check why validation RMSE is this large
model_p.get_modeling_result()

training size: 6400
validation size: 1600
test size: 2000
categorical binary vars:  50
categorical non binary vars:  7
continues vars:  29
training RMSE: 202.3932643418208
valiation RMSE: 18261.855187229106


(202.3932643418208, 18261.855187229106)

In [8]:
### ridge with continuous variables (with normalization)
variables = continous_vars
model_p = modeling_pipeline(data, Ridge(), variables)
categorical_vars, categorical_binary_vars, continous_vars = model_p.divide_variables()
train_data, val_data, test_data = model_p.split_data()
X_train, y_train, X_val, y_val, X_test, y_test = model_p.get_X_y()
X_train_normalized, X_val_normalized, X_test_normalized = model_p.get_normalized_X_y()
model_p.get_normalized_modeling_result()

categorical binary vars:  50
categorical non binary vars:  7
continues vars:  29
training size: 6400
validation size: 1600
test size: 2000
training RMSE: 209.4713892099817
valiation RMSE: 18263.815095915033


(209.4713892099817, 18263.815095915033)

In [9]:
# implement XGboost
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

variables = [col for col in data.columns.unique().tolist() if col not in ['price_usd','date_time']]
model_p = modeling_pipeline(data, xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10), variables)
train_data, val_data, test_data = model_p.split_data()
X_train, y_train, X_val, y_val, X_test, y_test = model_p.get_X_y()
model_p.get_modeling_result()

training size: 6400
validation size: 1600
test size: 2000


/anaconda3/envs/leetcode/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/envs/leetcode/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


training RMSE: 162.80642533154264
valiation RMSE: 18264.444565670146


(162.80642533154264, 18264.444565670146)